# 배터리 데이터 처리 시스템 (Refactored)

이 노트북은 `dataprocess.py` 모듈을 사용하여 데이터를 로드하고 분석합니다.
디버깅 및 시각화를 위해 대화형으로 실행할 수 있습니다.

In [4]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import dataprocess
import importlib

# 모듈 수정 시 재로딩을 위해
importlib.reload(dataprocess)

# 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

print("✅ 라이브러리 및 모듈 로딩 완료")

✅ 라이브러리 및 모듈 로딩 완료


In [5]:
# 분석할 경로 리스트 (사용자가 수정)
paths = [
    r"C:\Users\Ryu\Python_project\data\dataprocess\Rawdata\A1_MP1_4500mAh_T23_1",
    # 추가 경로를 여기에 입력하세요
    
]

print(f"분석 대상 경로 개수: {len(paths)}")

분석 대상 경로 개수: 1


In [3]:
# 데이터 처리 파이프라인 실행
df_results, loaded_data = dataprocess.process_battery_data(paths)

# 결과 요약 확인
display(df_results[['folder_name', 'cycler_type', 'capacity_mAh']])

🔋 배터리 데이터 처리 파이프라인 시작

[1/1] 처리 중: A1_MP1_4500mAh_T23_1
----------------------------------------------------------------------
  📁 폴더명: A1_MP1_4500mAh_T23_1
  🔧 사이클러 타입: PNE
  ⚡ 용량: 4500.0 mAh
  📊 발견된 채널: 2개
    - M02Ch073[073] 로딩 중...
      ✓ 사이클 데이터: 4,887행
      ✓ 프로파일 데이터: 572,187행
    - M02Ch074[074] 로딩 중...
      ✓ 사이클 데이터: 4,887행
      ✓ 프로파일 데이터: 572,433행

✅ 데이터 처리 완료


,folder_name,cycler_type,capacity_mAh
0,A1_MP1_4500mAh_T23_1,PNE,4500.0


## 로드된 데이터 상세 탐색

In [ ]:
# loaded_data 상세 탐색
print("="*70)
print("📊 loaded_data 상세 정보")
print("="*70)

# 1. 전체 구조 확인
print("\n🔍 1. 전체 구조:")
for data_type, data_dict in loaded_data.items():
    print(f"  {data_type}: {len(data_dict)}개 채널")
    if data_dict:
        for key in data_dict.keys():
            print(f"    - {key}")

# 2. PNE 사이클 데이터 상세 정보
if loaded_data['pne_cycle']:
    print("\n" + "="*70)
    print("📌 2. PNE 사이클 데이터 상세 정보")
    print("="*70)
    for key, df in loaded_data['pne_cycle'].items():
        print(f"\n채널: {key}")
        print(f"  - 행 개수: {len(df):,}")
        print(f"  - 열 개수: {len(df.columns)}")
        print(f"  - 열 이름: {list(df.columns)}")
        print(f"  - 메모리 사용량: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
        if 'Cycle' in df.columns:
             print(f"  - 사이클 범위: {df['Cycle'].min()} ~ {df['Cycle'].max()}")
        
        print(f"\n  통계 정보:")
        if 'DchgCap_mAh' in df.columns:
            print(f"    방전용량 평균: {df['DchgCap_mAh'].mean():.2f} mAh")
            print(f"    방전용량 최대: {df['DchgCap_mAh'].max():.2f} mAh")
            print(f"    방전용량 최소: {df['DchgCap_mAh'].min():.2f} mAh")
        if 'Temp_C' in df.columns:
            print(f"    온도 평균: {df['Temp_C'].mean():.2f} °C")

# 3. PNE 프로파일 데이터 상세 정보
if loaded_data['pne_profile']:
    print("\n" + "="*70)
    print("📌 3. PNE 프로파일 데이터 상세 정보")
    print("="*70)
    for key, df in loaded_data['pne_profile'].items():
        print(f"\n채널: {key}")
        print(f"  - 행 개수: {len(df):,}")
        print(f"  - 열 개수: {len(df.columns)}")
        print(f"  - 열 이름: {list(df.columns)}")
        print(f"  - 메모리 사용량: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
        if 'voltage_v' in df.columns:
            print(f"    전압 평균: {df['voltage_v'].mean():.2f} V")
            print(f"    전압 범위: {df['voltage_v'].min():.2f} ~ {df['voltage_v'].max():.2f} V")
        if 'current_mA' in df.columns:
            print(f"    전류 평균: {df['current_mA'].mean():.2f} mA")

# 4. Toyo 데이터 상세 정보
if loaded_data['toyo_cycle']:
    print("\n" + "="*70)
    print("📌 4. Toyo 사이클 데이터 상세 정보")
    print("="*70)
    for key, df in ak['toyo_cycle'].items():
        print(f"\n채  널: {key}")
        print(f"  - 행 개수: {len(df):,}")
        print(f"  - 열 개수: {len(df.columns)}")
        print(f"  - 열 이름: {list(df.columns)}")
        if 'Cycle' in df.columns:
             print(f"  - 사이클 범위: {df['Cycle'].min()} ~ {df['Cycle'].max()}")

if loaded_data['toyo_profile']:
    print("\n" + "="*70)
    print("📌 5. Toyo 프로파일 데이터 상세 정보")
    print("="*70)
    for key, df in loaded_data['toyo_profile'].items():
        print(f"\n채널: {key}")
        print(f"  - 행 개수: {len(df):,}")
        print(f"  - 열 개수: {len(df.columns)}")
        print(f"  - 열 이름: {list(df.columns)}")

print("\n" + "="*70)

📊 loaded_data 상세 정보

🔍 1. 전체 구조:
  pne_cycle: 2개 채널
    - A1_MP1_4500mAh_T23_1_M02Ch073[073]
    - A1_MP1_4500mAh_T23_1_M02Ch074[074]
  pne_profile: 2개 채널
    - A1_MP1_4500mAh_T23_1_M02Ch073[073]
    - A1_MP1_4500mAh_T23_1_M02Ch074[074]
  toyo_cycle: 0개 채널
  toyo_profile: 0개 채널

📌 2. PNE 사이클 데이터 상세 정보

채널: A1_MP1_4500mAh_T23_1_M02Ch073[073]
  - 행 개수: 4,887
  - 열 개수: 13
  - 열 이름: ['Cycle', 'Condition', 'ChgCap_mAh', 'DchgCap_mAh', 'OCV_mV', 'imp', 'VoltageMax_mV', 'ChgPow_mW', 'DchgPow_mW', 'Steptime_s', 'Temp_C', 'EndState', 'Current_mA']
  - 메모리 사용량: 0.48 MB
  - 사이클 범위: 1 ~ 601

  통계 정보:
    방전용량 평균: 1035.78 mAh
    방전용량 최대: 4649.32 mAh
    방전용량 최소: 0.00 mAh
    온도 평균: 23.68 °C

채널: A1_MP1_4500mAh_T23_1_M02Ch074[074]
  - 행 개수: 4,887
  - 열 개수: 13
  - 열 이름: ['Cycle', 'Condition', 'ChgCap_mAh', 'DchgCap_mAh', 'OCV_mV', 'imp', 'VoltageMax_mV', 'ChgPow_mW', 'DchgPow_mW', 'Steptime_s', 'Temp_C', 'EndState', 'Current_mA']
  - 메모리 사용량: 0.48 MB
  - 사이클 범위: 1 ~ 601

  통계 정보:
    방전용량 평균: 1037.2